In [1]:
import gpxpy
import gpxpy.gpx
import folium
import time


import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.right'] = False
from IPython.display import display

In [2]:
with open('/Users/mark/Downloads/That_Ride.gpx', 'r') as gpx_file:
    gpx_his = gpxpy.parse(gpx_file)

In [3]:
with open('/Users/mark/Downloads/cocking_edited.gpx', 'r') as gpx_file:
    gpx_mine = gpxpy.parse(gpx_file)

In [4]:
with open('/Users/mark/Downloads/rob_ride.gpx', 'r') as gpx_file:
    gpx_rob = gpxpy.parse(gpx_file)

In [5]:
my_route = []

for track in gpx_mine.tracks:
    for segment in track.segments:
        for point in segment.points:
            my_route.append({
                'time': point.time,
                'latitude': point.latitude,
                'longitude': point.longitude,
                'elevation': point.elevation
            })
            
his_route = []

for track in gpx_his.tracks:
    for segment in track.segments:
        for point in segment.points:
            his_route.append({
                'time': point.time,
                'latitude': point.latitude,
                'longitude': point.longitude,
                'elevation': point.elevation
            })

rob_route = []

for track in gpx_rob.tracks:
    for segment in track.segments:
        for point in segment.points:
            rob_route.append({
                'time': point.time,
                'latitude': point.latitude,
                'longitude': point.longitude,
                'elevation': point.elevation
            })

In [6]:
df_mine = pd.DataFrame(my_route)
df_his = pd.DataFrame(his_route)
df_rob = pd.DataFrame(rob_route)

df_mine.set_index('time', inplace=True)
df_his.set_index('time', inplace=True)
df_rob.set_index('time', inplace=True)

In [7]:
df_mine['timestr'] = df_mine.index.strftime('%H.%M:%S')
df_his['timestr'] = df_his.index.strftime('%H.%M:%S')
df_rob['timestr'] = df_rob.index.strftime('%H.%M:%S')

In [8]:
n_track=len(df_mine.between_time('10:36:05','10:36:19'))

In [9]:
route_map = folium.Map(
    location=[50.9502793, -0.7530442],
    zoom_start=15,
    tiles='OpenStreetMap',
    width=1024,
    height=600
)

for _, row in df_mine.between_time('10:36:05','10:36:19').iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=3,
        popup=[row['timestr']],
    ).add_to(route_map)

for _, row in df_his.between_time('10:36:05','10:36:19').iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=3,
        color='red',
        popup=[row['timestr']],
    ).add_to(route_map)

for _, row in df_rob.between_time('10:36:05','10:36:19').iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=3,
        color='green',
        popup=[row['timestr']],
    ).add_to(route_map)
display(route_map)


In [193]:
route_map = folium.Map(
    location=[50.9502793, -0.7530442],
    zoom_start=15,
    tiles='OpenStreetMap',
    width=1024,
    height=600
)


coordinates = [tuple(x) for x in df_mine.between_time('10:36:13','10:36:19')[['latitude', 'longitude']].to_numpy()]
folium.PolyLine(coordinates, weight=4).add_to(route_map)

coordinates = [tuple(x) for x in df_his.between_time('10:36:13','10:36:19')[['latitude', 'longitude']].to_numpy()]
folium.PolyLine(coordinates, weight=4, color='red').add_to(route_map)

coordinates = [tuple(x) for x in df_rob.between_time('10:36:13','10:36:19')[['latitude', 'longitude']].to_numpy()]
folium.PolyLine(coordinates, weight=4, color='green').add_to(route_map)

display(route_map)

In [154]:
for _, row in df_mine.between_time('10:36:10','10:36:19').iterrows():
    print (row.index)

Index(['latitude', 'longitude', 'elevation'], dtype='object')
Index(['latitude', 'longitude', 'elevation'], dtype='object')
Index(['latitude', 'longitude', 'elevation'], dtype='object')
Index(['latitude', 'longitude', 'elevation'], dtype='object')
Index(['latitude', 'longitude', 'elevation'], dtype='object')
Index(['latitude', 'longitude', 'elevation'], dtype='object')
Index(['latitude', 'longitude', 'elevation'], dtype='object')
Index(['latitude', 'longitude', 'elevation'], dtype='object')


In [135]:
df_mine.between_time('10:36:10','10:36:19')

,latitude,longitude,elevation
time,,,
2021-11-21 10:36:10+00:00,50.949019,-0.752385,-16.8
2021-11-21 10:36:11+00:00,50.949199,-0.752398,-17.8
2021-11-21 10:36:12+00:00,50.949376,-0.752413,-18.8
2021-11-21 10:36:13+00:00,50.949546,-0.752395,-19.6
2021-11-21 10:36:14+00:00,50.949703,-0.752337,-20.0
2021-11-21 10:36:16+00:00,50.949962,-0.752115,-19.0
2021-11-21 10:36:17+00:00,50.950079,-0.751985,-18.8
2021-11-21 10:36:19+00:00,50.950295,-0.751787,-19.2


In [152]:
df_mine.index.strftime('%H.%M:%S')

Index(['10.35:37', '10.35:39', '10.35:41', '10.35:42', '10.35:43', '10.35:44',
       '10.35:46', '10.35:47', '10.35:48', '10.35:49', '10.35:50', '10.35:51',
       '10.35:52', '10.35:53', '10.35:54', '10.35:55', '10.35:56', '10.35:57',
       '10.35:58', '10.35:59', '10.36:00', '10.36:01', '10.36:02', '10.36:03',
       '10.36:04', '10.36:05', '10.36:06', '10.36:07', '10.36:08', '10.36:09',
       '10.36:10', '10.36:11', '10.36:12', '10.36:13', '10.36:14', '10.36:16',
       '10.36:17', '10.36:19', '10.36:20', '10.36:21', '10.36:23', '10.36:30'],
      dtype='object', name='time')

In [200]:
route_map = folium.Map(
    location=[50.9502793, -0.7530442],
    zoom_start=15,
    tiles='OpenStreetMap',
    width=1024,
    height=600
)

for i in range(n_track):
    folium.CircleMarker(
        location=[df_mine.iloc[i]['latitude'], df_mine.iloc[i]['longitude']],
        radius=3,
        popup=[df_mine.iloc[i]['timestr']],
    ).add_to(route_map)
#     time.sleep(1)
display(route_map)

In [187]:
df_mine.iloc[0]['latitude']

50.944559